In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Experiment 1 - Simple Logistic Regression

In [2]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

def evaluate_esa_metrics(esa_baseline, esa_moral, tau):
    return {
        'esa_diff': esa_difference(esa_baseline, esa_moral),
        'tcr': threshold_crossing_rate(esa_baseline, esa_moral, tau),
        'moral_win_rate': moral_win_rate(esa_baseline, esa_moral)
    }

In [4]:
if __name__ == '__main__':
    #read orig dataset
    df = pd.read_csv('credit_risk_dataset.csv')
    
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()
    
    # Apply label encoding
    df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
    df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
    df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
    df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])
    
    y = df['loan_status']
    X = df.drop(columns=['loan_status'])

    n_splits = 20
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    preds = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        preds[val_idx] = y_val_pred

        print(f"Fold {fold+1} Baseline Accuracy:", accuracy_score(y_val, y_val_pred))

    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time
    
    print("\nLogistic Regression Baseline")
    print("Baseline Accuracy:", accuracy_score(y, preds))
    print(f"Time taken: {elapsed_time:.6f} seconds")
    
    metrics = evaluate_classification(y, preds)
    print(metrics)

Fold 1 Baseline Accuracy: 0.811042944785276
Fold 2 Baseline Accuracy: 0.7937384898710865
Fold 3 Baseline Accuracy: 0.805402087170043
Fold 4 Baseline Accuracy: 0.8047882136279927
Fold 5 Baseline Accuracy: 0.8004910988336402
Fold 6 Baseline Accuracy: 0.8029465930018416
Fold 7 Baseline Accuracy: 0.8103130755064457
Fold 8 Baseline Accuracy: 0.807243707796194
Fold 9 Baseline Accuracy: 0.809085328422345
Fold 10 Baseline Accuracy: 0.809085328422345
Fold 11 Baseline Accuracy: 0.7961939840392879
Fold 12 Baseline Accuracy: 0.8035604665438919
Fold 13 Baseline Accuracy: 0.8041743400859422
Fold 14 Baseline Accuracy: 0.809085328422345
Fold 15 Baseline Accuracy: 0.8041743400859422
Fold 16 Baseline Accuracy: 0.8011049723756906
Fold 17 Baseline Accuracy: 0.7992633517495396
Fold 18 Baseline Accuracy: 0.8084714548802947
Fold 19 Baseline Accuracy: 0.7986494782074892
Fold 20 Baseline Accuracy: 0.8066298342541437

Logistic Regression Baseline
Baseline Accuracy: 0.8042724287161229
Time taken: 1.779093 second

In [5]:
preds

array([1., 0., 1., ..., 1., 0., 0.])

In [6]:
np.savetxt('out.txt', y_val, delimiter=',')

# Experiment 2 - Logistic Regression on ESA-augmented Dataset 

In [7]:
if __name__ == '__main__':
    #read orig dataset
    df2 = pd.read_csv('credit_risk_dataset - ethics.csv')
    
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()
    
    # Apply label encoding
    df2['person_home_ownership'] = label_encoder.fit_transform(df2['person_home_ownership'])
    df2['loan_intent'] = label_encoder.fit_transform(df2['loan_intent'])
    df2['loan_grade'] = label_encoder.fit_transform(df2['loan_grade'])
    df2['cb_person_default_on_file'] = label_encoder.fit_transform(df2['cb_person_default_on_file'])
    
    y = df2['loan_status']
    X = df2.drop(columns=['loan_status'])

    n_splits = 20
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    preds = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)

        y_val_pred = model.predict(X_val)
        preds[val_idx] = y_val_pred

        print(f"Fold {fold+1} ESA-Augmented Accuracy:", accuracy_score(y_val, y_val_pred))

    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time
    
    print("\nLogistic Regression ESA-Augmented")
    print("Accuracy:", accuracy_score(y, preds))
    print(f"Time taken: {elapsed_time:.6f} seconds")
    
    metrics = evaluate_classification(y, preds)
    print(metrics)

Fold 1 ESA-Augmented Accuracy: 0.811042944785276
Fold 2 ESA-Augmented Accuracy: 0.7937384898710865
Fold 3 ESA-Augmented Accuracy: 0.805402087170043
Fold 4 ESA-Augmented Accuracy: 0.8047882136279927
Fold 5 ESA-Augmented Accuracy: 0.8004910988336402
Fold 6 ESA-Augmented Accuracy: 0.8029465930018416
Fold 7 ESA-Augmented Accuracy: 0.8103130755064457
Fold 8 ESA-Augmented Accuracy: 0.807243707796194
Fold 9 ESA-Augmented Accuracy: 0.809085328422345
Fold 10 ESA-Augmented Accuracy: 0.809085328422345
Fold 11 ESA-Augmented Accuracy: 0.7961939840392879
Fold 12 ESA-Augmented Accuracy: 0.8035604665438919
Fold 13 ESA-Augmented Accuracy: 0.8041743400859422
Fold 14 ESA-Augmented Accuracy: 0.809085328422345
Fold 15 ESA-Augmented Accuracy: 0.8041743400859422
Fold 16 ESA-Augmented Accuracy: 0.8011049723756906
Fold 17 ESA-Augmented Accuracy: 0.7992633517495396
Fold 18 ESA-Augmented Accuracy: 0.8084714548802947
Fold 19 ESA-Augmented Accuracy: 0.7986494782074892
Fold 20 ESA-Augmented Accuracy: 0.806629834254

# Experiment 3 - Logistic Regression on ESA-augmented Dataset with CatBoost

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

In [9]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

def evaluate_regression(y_true, y_pred):
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'rmse': mean_squared_error(y_true, y_pred, squared=False)
    }

def evaluate_esa_metrics(esa_baseline, esa_moral, tau):
    return {
        'esa_diff': esa_difference(esa_baseline, esa_moral),
        'tcr': threshold_crossing_rate(esa_baseline, esa_moral, tau),
        'moral_win_rate': moral_win_rate(esa_baseline, esa_moral)
    }

In [10]:
def evaluate_esa_decision_function(model, X, phi_matrix, alpha, tau_values):
    y_proba = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)
    esa_scores = np.array([esa_score(phi, alpha) for phi in phi_matrix])
    loan_status_moral = (esa_scores >= tau_values).astype(int)

    eval_moral = evaluate_classification(y_pred, loan_status_moral)
    eval_moral['roc_auc'] = roc_auc_score(y_pred, loan_status_moral)
    return eval_moral, esa_scores, loan_status_moral

In [11]:
if __name__ == '__main__':
    # Example dummy ESA weights
    alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])  # For consequentialism

    # Load data (replace this with your actual DataFrame)
    df = pd.read_csv('credit_risk_dataset - ethics.csv')  # Assumes 6 ESA features + CST + loan_status + other inputs

    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Apply label encoding
    df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
    df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
    df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
    df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])
    
    # Extract ESA features and CST
    esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
    tau_values = df['CST'].values
    y = df['loan_status']
    X = df.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])

    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    catboost_params = {
        'depth': 7,
        'learning_rate': 0.19893301995319765,
        'bagging_temperature': 0.7979373495258176,
        'l2_leaf_reg': 5,
        'loss_function': 'Logloss',
        'iterations': 400,
        'grow_policy': 'Lossguide',
        'eval_metric': 'AUC',
        'verbose': False
    }

    preds = np.zeros(len(X))
    moral_preds = np.zeros(len(X))
    all_esa = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        phi_val = esa_features[val_idx]
        tau_val = tau_values[val_idx]

        model = CatBoostClassifier(**catboost_params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)

        # Apply ESA-informed decision override
        eval_moral, esa_vals, moral_decisions = evaluate_esa_decision_function(model, X_val, phi_val, alpha, tau_val)
        preds[val_idx] = model.predict(X_val)
        moral_preds[val_idx] = moral_decisions

        print(f"Fold {fold+1}:", eval_moral)
        
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

    print("\n--- Final Evaluation ---")
    print("Standard Accuracy:", accuracy_score(y, preds))
    print("Moral Accuracy:", accuracy_score(y, moral_preds))
    print("Moral Agreement:", accuracy_score(preds, moral_preds))

    
metrics = evaluate_classification(y, preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      5457
           1       0.27      0.38      0.31      1060

    accuracy                           0.73      6517
   macro avg       0.57      0.59      0.57      6517
weighted avg       0.77      0.73      0.75      6517


Confusion Matrix:
[[4349 1108]
 [ 657  403]]
Fold 1: {'accuracy': 0.7291698634340954, 'precision': 0.26671078755790867, 'recall': 0.38018867924528305, 'f1_score': 0.31349669389342666, 'roc_auc': 0.588573357397976}

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      5441
           1       0.27      0.38      0.32      1075

    accuracy                           0.73      6516
   macro avg       0.57      0.59      0.57      6516
weighted avg       0.77      0.73      0.75      6516


Confusion Matrix:
[[4351 1090]
 [ 666  409]]
Fold 2: {'accuracy': 0.73050951503990

# Experiment 4 - Logistic Regression on regular Dataset with CatBoost

In [12]:
if __name__ == '__main__':
    # Example dummy ESA weights
    #alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])  # For consequentialism

    # Load data (replace this with your actual DataFrame)
    df4 = pd.read_csv('credit_risk_dataset.csv')  # Assumes 6 ESA features + CST + loan_status + other inputs

    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Apply label encoding
    df4['person_home_ownership'] = label_encoder.fit_transform(df4['person_home_ownership'])
    df4['loan_intent'] = label_encoder.fit_transform(df4['loan_intent'])
    df4['loan_grade'] = label_encoder.fit_transform(df4['loan_grade'])
    df4['cb_person_default_on_file'] = label_encoder.fit_transform(df4['cb_person_default_on_file'])
    
    # Extract ESA features and CST
    #esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
    #tau_values = df['CST'].values
    y = df['loan_status']
    X = df.drop(columns=['loan_status'])

    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    catboost_params = {
        'depth': 7,
        'learning_rate': 0.19893301995319765,
        'bagging_temperature': 0.7979373495258176,
        'l2_leaf_reg': 5,
        'loss_function': 'Logloss',
        'iterations': 400,
        'grow_policy': 'Lossguide',
        'eval_metric': 'AUC',
        'verbose': False
    }

    preds = np.zeros(len(X))
    moral_preds = np.zeros(len(X))
    all_esa = np.zeros(len(X))

    #start timer
    start_time = time.perf_counter()
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        phi_val = esa_features[val_idx]
        tau_val = tau_values[val_idx]

        model = CatBoostClassifier(**catboost_params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=10)

        # Apply ESA-informed decision override
        #eval_moral, esa_vals, moral_decisions = evaluate_esa_decision_function(model, X_val, phi_val, alpha, tau_val)
        preds[val_idx] = model.predict(X_val)
        #moral_preds[val_idx] = moral_decisions

        #print(f"Fold {fold+1}:", eval_moral)
        
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

    print("\n--- Final Evaluation ---")
    print("Standard Accuracy:", accuracy_score(y, preds))
    print("Moral Accuracy:", accuracy_score(y, moral_preds))
    print("Moral Agreement:", accuracy_score(preds, moral_preds))

    
metrics = evaluate_classification(y, preds)
print(metrics)


--- Final Evaluation ---
Standard Accuracy: 0.936527423958749
Moral Accuracy: 0.7818360394094718
Moral Agreement: 0.8371443479328443

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     25473
           1       0.97      0.73      0.83      7108

    accuracy                           0.94     32581
   macro avg       0.95      0.86      0.90     32581
weighted avg       0.94      0.94      0.93     32581


Confusion Matrix:
[[25340   133]
 [ 1935  5173]]
{'accuracy': 0.936527423958749, 'precision': 0.974934036939314, 'recall': 0.727771525042206, 'f1_score': 0.8334138875463188, 'roc_auc': 0.8612751552114025}


# Experiment 5 - With Penalized Term

In [17]:
 preds_penalized = np.zeros(len(X))

#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]

    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    #sample_weights = np.clip(1 + 7 * moral_penalty, 1, 20)
    sample_weights = 1 + 7 * moral_penalty
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train, sample_weight=sample_weights)

    y_val_pred = model.predict(X_val)
    preds_penalized[val_idx] = y_val_pred

    print(f"Fold {fold+1} ESA Penalized Accuracy:", accuracy_score(y_val, y_val_pred))
    
    #end time
    end_time = time.perf_counter()
    
    elapsed_time = end_time - start_time

print("\n--- Final Evaluation (Logistic Regression ESA Penalized) ---")
print("ESA Penalized Accuracy:", accuracy_score(y, preds_penalized))
print(f"Time taken: {elapsed_time:.6f} seconds")
    

metrics = evaluate_classification(y_val, y_val_pred)
print(metrics)


Fold 1 ESA Penalized Accuracy: 0.8037440540125825
Fold 2 ESA Penalized Accuracy: 0.8031000613873542
Fold 3 ESA Penalized Accuracy: 0.8052486187845304
Fold 4 ESA Penalized Accuracy: 0.8049416820135052
Fold 5 ESA Penalized Accuracy: 0.8032535297728668

--- Final Evaluation (Logistic Regression ESA Penalized) ---
ESA Penalized Accuracy: 0.8040575795709156
Time taken: 0.468315 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.98      0.89      5095
           1       0.72      0.16      0.26      1421

    accuracy                           0.80      6516
   macro avg       0.76      0.57      0.58      6516
weighted avg       0.79      0.80      0.75      6516


Confusion Matrix:
[[5003   92]
 [1190  231]]
{'accuracy': 0.8032535297728668, 'precision': 0.7151702786377709, 'recall': 0.1625615763546798, 'f1_score': 0.26490825688073394, 'roc_auc': 0.572252328903542}


# Override later decisions

In [16]:
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])
esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df['CST'].values
preds_override = np.zeros(len(X))

#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # ESA override decision logic
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_decisions = (esa_vals >= tau_val).astype(int)
    preds_override[val_idx] = moral_decisions

    print(f"Fold {fold+1} ESA Override Accuracy:", accuracy_score(y_val, moral_decisions))

#end time
end_time = time.perf_counter()
elapsed_time = end_time - start_time

print("\n--- Final Evaluation (Logistic Regression ESA Override) ---")
print("ESA Override Accuracy:", accuracy_score(y, preds_override))
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y, preds_override)
print(metrics)

Fold 1 ESA Override Accuracy: 0.6926499923277581
Fold 2 ESA Override Accuracy: 0.6959791282995703
Fold 3 ESA Override Accuracy: 0.7004297114794352
Fold 4 ESA Override Accuracy: 0.6955187231430325
Fold 5 ESA Override Accuracy: 0.6887661141804788

--- Final Evaluation (Logistic Regression ESA Override) ---
ESA Override Accuracy: 0.6946686719253553
Time taken: 0.227110 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.80      0.80     25473
           1       0.31      0.32      0.31      7108

    accuracy                           0.69     32581
   macro avg       0.56      0.56      0.56     32581
weighted avg       0.70      0.69      0.70     32581


Confusion Matrix:
[[20377  5096]
 [ 4852  2256]]
{'accuracy': 0.6946686719253553, 'precision': 0.3068552774755169, 'recall': 0.317388857625211, 'f1_score': 0.31203319502074683, 'roc_auc': 0.5586669487356613}
